In [ ]:
from tkinter import *
from tkinter import messagebox
import time
tic=time.perf_counter()
#
def Simplicar_Lista(x): 
    Lista_Simplificada = []
    for a in x:
        Lista_Simplificada.extend(x[a])
        
    return Lista_Simplificada
#
def mult_terms(x,y): #Multiplica dos terminos
    resultado = []
    for a in x:
        for b in y:
            ValorMulti = mult_terms(a,b) #Recursivo
            resultado.append(ValorMulti) if len(ValorMulti) != 0 else None #Agrega el valor a el resultado final
    return resultado
#
def Minterminos_mezclados(a): # Busca los minterminos mezclados 
    guiones = a.count('-') #Cuenta los guiones
    if guiones == 0:
        return [str(int(a,2))] #Si no hay regresa
    x = [bin(i)[2:].zfill(guiones) for i in range(pow(2,guiones))] 
    t1 = []
    for i in range(pow(2,guiones)):
        t2,ind = a[:],-1
        for j in x[0]:
            if ind != -1:
                ind = ind+t2[ind+1:].find('-')+1
            else:
                ind = t2[ind+1:].find('-')
            t2 = t2[:ind]+j+t2[ind+1:]
        t1.append(str(int(t2,2)))
        x.pop(0)
    return t1
#
def Quitar_dc(listaI,listaF): # Quita los dc
    r = []
    for a in listaI:
        if int(a) not in listaF:
            r.append(a) #Si no estan los agraga
    return r
#
def Bit_Distinos(a,b): # Busca si dos minterminos son diferentes en un bit
    bit = 0
    for z in range(len(a)):
        if a[z] != b[z]: #Si son distintos
            diferencia = z
            bit += 1
            if bit>1:
                return (False,None)
    return (True,diferencia)
#
def PIE(x): # Encuenta los PIE
    r = []
    for a in x:
        if len(x[a]) == 1:
            r.append(x[a][0]) if x[a][0] not in r else None #Agrega los PIE, si no están no hace nada
    return r
#
def Elim_terminos(_chart,terminos): # Quita los minterminos ya seleccionados 
    for a in terminos:
        for b in Minterminos_mezclados(a):
            try:
                del _chart[b]
            except KeyError:
                pass 
#
def Normales_Negados(x): # Omite los - y pone si es necesario los negados 
    variables = []
    for a in range(len(x)):
        if x[a] == '0':
            variables.append(chr(a+65)+"'") #Agrega el negado
        elif x[a] == '1':
            variables.append(chr(a+65)) #Solo lo agrega
    return variables

def Respuesta_Final(FinalF,tiempo):
    #toc=time.perf_counter()
    #tiempo = str(toc - tic:0.4f)
    messagebox.showinfo("Simplificación", "La solución es:" + " " + FinalF + "\n Tiempo de ejecución: " + tiempo + " s")
#---------------------------------------------------------------------------------------------------

def inicio():
    minterminos = [int(i) for i in MTS.get().strip().split()]  # Agrupa los minterminos en la lista minterminos
    dc = []
    minterminos.sort()
    size = len(bin(minterminos[-1]))-2 # bin pasa a binario, len = longitud de la lista
    ap = set() # Set = lista no ordenada que no se puede cambiar e ignora a los duplicados
    grupos = {}  

#-----------------------------------------------------------------------------------------------------
# Primera tabla 
    for mt in minterminos:
        try:
            grupos[bin(mt).count('1')].append(bin(mt)[2:].zfill(size))  #count cuenta los unos, append = agraga el mintermino a la lista grupos 
        except KeyError:
            grupos[bin(mt).count('1')] = [bin(mt)[2:].zfill(size)] 

#--------------------------------------------------------------------------------------------------------
# Tabla 2 e implicantes primos
    while True:
        tmp = grupos.copy()
        bandera = True
        P = set()
        grupos = {}
        J = 0
        l = sorted(list(tmp.keys()))
        for i in range(len(l)-1):
            for j in tmp[l[i]]: # Va de uno en uno en el primer grupo de elementos 
                for k in tmp[l[i+1]]: # Va de uno en uno en el segundo grupo de elementos
                    res = Bit_Distinos(j,k) # Compara j y k de las listas pasadas para ver cuantos difieren
                    if res[0]: # Si son diferentes en solo un mintermino entra en el try
                        try:
                            grupos[J].append(j[:res[1]]+'-'+j[res[1]+1:]) if j[:res[1]]+'-'+j[res[1]+1:] not in grupos[J] else None # Pone un - en el bit que cambia
                        except KeyError:
                            grupos[J] = [j[:res[1]]+'-'+j[res[1]+1:]] # Se crea un nuevo grupo si no existe y - en el cambio
                        bandera = False
                        P.add(j) #  j marcando cada uno
                        P.add(k) #  k marcado cada uno
            J += 1
        SM = set(Simplicar_Lista(tmp)).difference(P) # Desmarca los elemntos de cada tabla
        ap = ap.union(SM) # Se meten los implicantes primos a la lista

        if bandera: # Si ya no se pueden combinar más minterminos
            break

#--------------------------------------------------------------------------------------------------------------------------------
# Ultima tabla 
    S = len(str(minterminos[-1])) # mintermino con más digitos
    chart = {}
    for i in ap:
        minterminos_mezclados= Minterminos_mezclados(i)
        y = 0
        for j in Quitar_dc(minterminos_mezclados,dc):
            x = minterminos.index(int(j))*(S+1) # Aqui se ponen las x
            y = x+S
            try:
                chart[j].append(i) if i not in chart[j] else None # Se agregan los miterminos 
            except KeyError:
                chart[j] = [i]

    IPE = PIE(chart) # Funcion que busca los implicantes primos escenciales
    Elim_terminos(chart,IPE) #Se quirtan los PIE de las columnas relacionadas
#---------------------------------------------------------------------------------------------------------------------------------- 
    if(len(chart) == 0): # En el caso de que queden minterminos al quitar los PI
        respuesta = [Normales_Negados(i) for i in IPE] #Respuesta de los PIE
    else: #Caso contrartio 
        F = [[Normales_Negados(j) for j in chart[i]] for i in chart]
        while len(F)>1: # multiplicación continua 
            F[1] = mult_terms(F[0],F[1])
            F.pop(0)
        respuesta = [min(F[0],key=len)] # Terminos con menor cantidad de Fs
        respuesta.extend(Normales_Negados(i) for i in IPE) # Se agregan los PIE
    FinalF = (" + ".join(''.join(i) for i in respuesta))
    
    toc=time.perf_counter()
    tiempo = str(round(toc - tic,4))
    Respuesta_Final(FinalF,tiempo)
    
v1 = Tk()
v1.title("Simplificación de mintérminos")
v1.geometry("350x150")

N_variables = StringVar()
MTS = StringVar()
N_variables.set("4")
MTS.set("1 2 3 4")

e1 = Label(v1, text="Número de variables").place(x=10,y=10)
N_v = Entry(v1, textvariable=N_variables).place(x=170,y=10)

e2 = Label(v1, text="Introduzca cada mintérmino").place(x=10,y=40)
MTS_CAJA = Entry(v1, textvariable=MTS).place(x=170,y=40)

b1 = Button(v1, text="Simplificar mintérminos", command=inicio).place(x=100,y=100)

v1.mainloop()

# Prueba
